<a href="https://colab.research.google.com/github/Yun5141/comp0036/blob/master/FootballResultPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Introduction [*Terry*]

# 2. Data Import  [*Yun*]

We first import all the packages that will be used in the code:

In [11]:
import pandas as pd 

import numpy as np

And then the data sets from our github repository (https://github.com/Yun5141/comp0036):

In [12]:
# training data set
url="https://raw.githubusercontent.com/Yun5141/comp0036/master/data/epl-training.csv"
raw_training_data=pd.read_csv(url)

# test set
url = 'https://raw.githubusercontent.com/Yun5141/comp0036/master/data/epl-test.csv'
rawData_toPred = pd.read_csv(url)

# 2019 up-to-date data (from http://www.football-data.co.uk)
url = 'https://raw.githubusercontent.com/Yun5141/comp0036/master/data/epl2019.csv'
rawData_2019_uptodate = pd.read_csv(url)

# geometric information of teams
# to calculate the distance needed to travel for the away team
url = "https://raw.githubusercontent.com/Yun5141/comp0036/master/data/stadiums-with-GPS-coordinates.csv"
geometricData = pd.read_csv(url)

# 3. Data Transformation & Exploration [*Yun*]

### 3.1 Initial Data Exploration

We first visualize the raw training data and find that the shape is 4180 rows x 73 columns, but some columns are empty and unamed.

In [14]:
raw_training_data

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72
0,16/08/08,Arsenal,West Brom,1,0,H,1,0,H,H Webb,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16/08/08,Bolton,Stoke,3,1,H,3,0,H,C Foy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16/08/08,Everton,Blackburn,2,3,A,1,1,D,A Marriner,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16/08/08,Hull,Fulham,2,1,H,1,1,D,P Walton,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16/08/08,Middlesbrough,Tottenham,2,1,H,0,0,D,M Atkinson,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4175,12/05/2019,Liverpool,Wolves,2,0,H,1,0,H,M Atkinson,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4176,12/05/2019,Man United,Cardiff,0,2,A,0,1,A,J Moss,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4177,12/05/2019,Southampton,Huddersfield,1,1,D,1,0,H,L Probert,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4178,12/05/2019,Tottenham,Everton,2,2,D,1,0,H,A Marriner,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


So we drop the unamed columns and find now the shape of the dataframe is 4180 rows x 22 columns.

In [36]:
raw_training_data = raw_training_data[raw_training_data.columns[~raw_training_data.columns.str.contains('Unnamed:')]]
raw_training_data

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,16/08/08,Arsenal,West Brom,1,0,H,1,0,H,H Webb,...,14,4,11,8,7,5,0,0,0,0
1,16/08/08,Bolton,Stoke,3,1,H,3,0,H,C Foy,...,8,2,13,12,4,3,1,2,0,0
2,16/08/08,Everton,Blackburn,2,3,A,1,1,D,A Marriner,...,5,11,11,9,3,5,2,2,0,0
3,16/08/08,Hull,Fulham,2,1,H,1,1,D,P Walton,...,6,6,10,9,5,6,3,0,0,0
4,16/08/08,Middlesbrough,Tottenham,2,1,H,0,0,D,M Atkinson,...,10,5,11,12,7,9,1,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4175,12/05/2019,Liverpool,Wolves,2,0,H,1,0,H,M Atkinson,...,5,2,3,11,4,1,0,2,0,0
4176,12/05/2019,Man United,Cardiff,0,2,A,0,1,A,J Moss,...,10,4,9,6,11,2,3,3,0,0
4177,12/05/2019,Southampton,Huddersfield,1,1,D,1,0,H,L Probert,...,3,3,8,6,4,3,0,1,0,0
4178,12/05/2019,Tottenham,Everton,2,2,D,1,0,H,A Marriner,...,3,9,10,13,7,4,0,2,0,0


##### 3.1.1 Invalid data

We then check if there are rows containing None, NaN, infinite or overflowed values:

In [41]:
def removeInvalidData(data):

    # remove data which contains None
    data.dropna(axis=0, how='any',inplace=True)

    # remove data which contains NaN, infinite or overflowed number 
    indices_to_keep = ~data.isin([np.nan, np.inf, -np.inf]).any(1)
    data = data[indices_to_keep]

    return data

assert raw_training_data.shape[0] == removeInvalidData(raw_training_data).shape[0]

The result indicates that there is no rows containing such values.

##### 3.1.2 Number of matches per season

##### 3.1.3 Win rate

##### 3.1.4 Relationship between attributes

### 3.2 Feature Construction

##### 3.2.1 Distance needed to travel for the away team

##### 3.2.2 Delta time from last match for home team and away team  

##### 3.2.3 Cumulative full time goal difference for home team and away team

##### 3.2.4 Averaged full time goal difference for home team and away team

##### 3.2.5 Performances of last 3 matches for home team and away team

### 3.3 Data Transformation

### 3.4 Second Data Exploration

# 4. Methodology Overview [*Yanke*]

# 5. Model Training & Validation [*Yanke*]

# 6. Result [*Yi*]

# 7. Final Predictions on Test Set [*Yusi*]